In [ ]:
# Trabalho 1 - Coleta, Preparação e Análise de Dados

## Webscrapping em ambiente real

### Objetivos da tarefa:

- Faça scraping para obter os 250 filmes com as maiores avaliações do IMDB. Devem
  ser obtidos: Título, Duração, url do poster e nota imdb.

- Faça scraping das páginas específicas dos 250 filmes obtidos no item anterior.
  Obtenha dessa página a popularidade e a listagem do elenco principal (incluindo nome
  do ator/atriz e da personagem).

- Salve as informações obtidas em arquivo json

import re
import json
from dataclasses import dataclas

from selenium import webdrivers
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
# abrindo o site do imdb
driver = webdriver.Firefox()
url = 'https://www.imdb.com'

driver.get(url)
# clicando no menu lateral
menu = driver.find_element(By.XPATH, r"//*[@id='imdbHeader-navDrawerOpen']")
menu.click()
# entrando na página dos 250 melhores filmes
top_250 = driver.find_element(By.LINK_TEXT, r'Top 250 Movies')
url = top_250.get_attribute("href")
url
driver.get(url)
# pega as tags de cada filme na página
tag_ul = driver.find_element(By.XPATH, r'//*[@id="__next"]/main/div/div[3]/section/div/div[2]/div/ul')
movies_list = tag_ul.find_elements(By.TAG_NAME, "li")
    
# exemplo de como é o texto de cada variável
print(movies_list[0].text)
# classe que monta a estrutura de dados para armazenar um filme
@dataclass
class Movie:
    title: str
    year: int
    duration: int #duração em minutos
    poster_url: str
    classification: str
    score: float

def movie_page_info(url):
    driver.get(url)
    
    driver.back()
    return popularity, cast_list
    
# método que passa por cada tag de filme na página da lista e cria um objeto Movie
def cria_filme(movie_tag):
    title = movie_tag.find_element(By.CLASS_NAME, "ipc-title__text").text # pegando o texto de classe "title"
    title = re.match(r'^\d+\. (.*)', title).group(1) # removendo o índice antes do nome do filme
    
    year = int(movie_tag.find_element(By.XPATH, r'./div[2]/div/div/div[2]/span[1]').text)
    
    time = re.match(r'(\d+)h (\d+)m', movie_tag.text.split('\n')[2])
    duration = int(time.group(1)) * 60 + int(time.group(2)) if time else None
    
    pagina = movie_tag.find_element(By.CLASS_NAME, "ipc-title-link-wrapper").get_attribute("href")
    
    score_tag = movie_tag.find_element(By.XPATH, ".//span[@class='ipc-rating-star--rating']")
    score = float(score_tag.text)
    
    url = movie_tag.find_element(By.XPATH, ".//div[2]/img").get_attribute("src")
    
    classification_tag = movie_tag.find_element(By.CLASS_NAME, "sc-b189961a-7")
    try:
        classification_tag = classification_tag.find_element(By.XPATH, ".//span[contains(@class, 'cli-title-metadata-item')][3]")
    except NoSuchElementException:
        classification_tag = None
    classification = classification_tag.text if classification_tag else "Unavailable"
        
    return Movie(title, year, duration, url, classification, score)

# Exemplo de objeto Movie
cria_filme(movies_list[1])
import json

movies_dict = []
for movie in movies_list:
    movies_dict.append(cria_filme(movie).__dict__)
    
print(movies_dict)
with open('top250_movies.json', 'w') as file:
    json.dump(movies_dict, file, indent=4)